In [14]:
import pandas as pd
import ETL
import importlib
importlib.reload(ETL)

pd.set_option('display.max_columns', None)

path = r"C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Raw"

# Banco de dados org 
df_org = pd.read_csv(path+"/BDorg/jogos.gz", compression="gzip")

# Kaggle Adão
df_KG_AD_jogos = pd.read_csv(path+"/kaggle_AD/jogos.csv", sep=",")
df_KG_AD_gols = pd.read_csv(path+"/kaggle_AD/gols.csv", sep=",")
df_KG_AD_estatisticas = pd.read_csv(path+"/kaggle_AD/estatisticas.csv", sep=",")
df_KG_AD_cartoes = pd.read_csv(path+"/kaggle_AD/cartoes.csv", sep=",")

#GH Hugo Brendow
df_GH_HB_jogos = pd.read_csv(path+"/GH_HugoBrendow/jogos.csv")

#GH Henrique
df_GH_Henr_jogos = pd.read_csv(path+"/GH_Henrique/jogos.csv")
df_GH_Henr_gols = pd.read_csv(path+"/GH_Henrique/gols.csv") 
df_GH_Henr_cartoes = pd.read_csv(path+"/GH_Henrique/cartoes.csv")

#GH Fabrimatt
df_GH_Fab_jogos = pd.read_excel(path+"/GH_Fabrimatt/jogos_stat.xlsx")

#GH Arruda
df_GH_Ard_jogos = ETL.unir_xlsx_em_df(path+"/GH_Arruda/dados/jogos")

##GH Avelar
#df_GH_Ave_jogos = json.load(path+"/GH_Avelar/2003/2003.json")


In [15]:
importlib.reload(ETL)
path = r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Raw\GH_Avelar\2003\2003.json'

df = ETL.ler_json(path)


# Criar DataFrame
df = pd.DataFrame(df['partidas'][0])

# Ajustar colunas aninhadas (opcional)
# Vamos expandir as colunas de pontuação geral em colunas separadas
def expandir_pontuacao(df, prefixo):
    pontuacao = pd.json_normalize(df[f'pontuacao_geral_{prefixo}'])
    pontuacao.columns = [f'{prefixo}_{col}' for col in pontuacao.columns]
    return pontuacao

pontuacao_mandante = expandir_pontuacao(df, 'mandante')
pontuacao_visitante = expandir_pontuacao(df, 'visitante')

# Juntar tudo em um único DataFrame
df_final = pd.concat([
    df.drop(['pontuacao_geral_mandante', 'pontuacao_geral_visitante'], axis=1),
    pontuacao_mandante,
    pontuacao_visitante
], axis=1)

# Mostrar as primeiras linhas
display(df_final.head(12))

,visitante,resultado,data_partida,placar_visitante,hora_partida,mandante,placar_mandante,estadio,mandante_gols_fora_casa,mandante_empates_fora_casa,mandante_total_jogos,mandante_gols_casa,mandante_jogos_fora_casa,mandante_vitorias_casa,mandante_derrotas_casa,mandante_total_pontos,mandante_empates_casa,mandante_pontos_fora_casa,mandante_total_gols_sofridos,mandante_total_vitorias,mandante_vitorias_fora_casa,mandante_total_derrotas,mandante_pontos_casa,mandante_derrotas_fora_casa,mandante_total_gols_marcados,mandante_jogos_casa,mandante_total_empates,visitante_gols_fora_casa,visitante_empates_fora_casa,visitante_total_jogos,visitante_gols_casa,visitante_jogos_fora_casa,visitante_vitorias_casa,visitante_derrotas_casa,visitante_total_pontos,visitante_empates_casa,visitante_pontos_fora_casa,visitante_total_gols_sofridos,visitante_total_vitorias,visitante_vitorias_fora_casa,visitante_total_derrotas,visitante_pontos_casa,visitante_derrotas_fora_casa,visitante_total_gols_marcados,visitante_jogos_casa,visitante_total_empates
0,Vasco,vitoria_mandante,29/03/2003,2,16h00,Guarani,4,Brinco de Ouro,0,0,1,4,0,1,0,3,0,0,2,1,0,0,3,0,4,1,0,2,0,1,0,1,0,0,0,0,0,4,0,0,1,0,1,2,0,0
1,Grêmio,vitoria_mandante,29/03/2003,0,16h00,Atlético pr,2,Arena da Baixada,0,0,1,2,0,1,0,3,0,0,0,1,0,0,3,0,2,1,0,0,0,1,0,1,0,0,0,0,0,2,0,0,1,0,1,0,0,0
2,Coritiba,empate,30/03/2003,1,16h00,Flamengo,1,Maracanã,0,0,1,1,0,0,0,1,1,0,1,0,0,0,1,0,1,1,1,1,1,1,0,1,0,0,1,0,1,1,0,0,0,0,0,1,0,1
3,Paysandu,empate,30/03/2003,2,16h00,Goiás,2,Serra Dourada,0,0,1,2,0,0,0,1,1,0,2,0,0,0,1,0,2,1,1,2,1,1,0,1,0,0,1,0,1,2,0,0,0,0,0,2,0,1
4,Ponte preta,empate,30/03/2003,1,16h00,Internacional,1,Beira Rio,0,0,1,1,0,0,0,1,1,0,1,0,0,0,1,0,1,1,1,1,1,1,0,1,0,0,1,0,1,1,0,0,0,0,0,1,0,1
5,Fluminense,vitoria_mandante,30/03/2003,0,16h00,Criciúma,2,Heriberto Hulse,0,0,1,2,0,1,0,3,0,0,0,1,0,0,3,0,2,1,0,0,0,1,0,1,0,0,0,0,0,2,0,0,1,0,1,0,0,0
6,São paulo,empate,30/03/2003,2,16h00,Juventude,2,Alfredo Jaconi,0,0,1,2,0,0,0,1,1,0,2,0,0,0,1,0,2,1,1,2,1,1,0,1,0,0,1,0,1,2,0,0,0,0,0,2,0,1
7,Bahia,empate,30/03/2003,0,16h00,Fortaleza,0,Castelão,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,1,0,1,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1
8,São caetano,empate,30/03/2003,2,16h00,Cruzeiro,2,Mineirão,0,0,1,2,0,0,0,1,1,0,2,0,0,0,1,0,2,1,1,2,1,1,0,1,0,0,1,0,1,2,0,0,0,0,0,2,0,1
9,Figueirense,empate,30/03/2003,1,16h00,Vitória,1,Barradão,0,0,1,1,0,0,0,1,1,0,1,0,0,0,1,0,1,1,1,1,1,1,0,1,0,0,1,0,1,1,0,0,0,0,0,1,0,1
